# Finding Similar Movies

We'll start by loading up the MovieLens dataset. Using Pandas, we can very quickly load the rows of the u.data and u.item files that we care about, and merge them together so we can work with movie names instead of ID's. (In a real production job, you'd stick with ID's and worry about the names at the display layer to make things more efficient. But this lets us understand what's going on better for now.)

In [19]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(movies, ratings)


In [20]:
ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


Now the amazing pivot_table function on a DataFrame will construct a user / movie rating matrix. Note how NaN indicates missing data - movies that specific users didn't rate.

In [33]:
movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's extract a Series of users who rated Toy Story:

In [40]:
toyStoryRating = movieRatings['Toy Story (1995)']
toyStoryRating.head()

user_id
0    NaN
1    5.0
2    4.0
3    NaN
4    NaN
Name: Toy Story (1995), dtype: float64

Pandas' corrwith function makes it really easy to compute the pairwise correlation of Toy Story' vector of user rating with every other movie! After that, we'll drop any results that have no data, and construct a new DataFrame of movies and their correlation score (similarity) to Toy Story:

In [45]:
import warnings #we can ignore this warning
similarMovies = movieRatings.corrwith(toyStoryRating)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head()

,0
title,
'Til There Was You (1997),0.534522
101 Dalmatians (1996),0.232118
12 Angry Men (1957),0.334943
187 (1997),0.651857
2 Days in the Valley (1996),0.162728


(That warning is safe to ignore.) Let's sort the results by similarity score, and we should have the movies most similar to Toy Story! Except... we don't. These results make no sense at all! This is why it's important to know your data - clearly we missed something important.

In [46]:
similarMovies.sort_values(ascending=False)

title
Ladybird Ladybird (1994)                                                             1.0
Wooden Man's Bride, The (Wu Kui) (1994)                                              1.0
Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)    1.0
Critical Care (1997)                                                                 1.0
Late Bloomers (1996)                                                                 1.0
                                                                                    ... 
Feast of July (1995)                                                                -1.0
Love and Death on Long Island (1997)                                                -1.0
Slingshot, The (1993)                                                               -1.0
Stalker (1979)                                                                      -1.0
Heavy (1995)                                                                        -1.0
Length: 1370, d

Our results are probably getting messed up by movies that have only been viewed by a handful of people who also happened to like Toy Story. So we need to get rid of movies that were only watched by a few people that are producing spurious results. Let's construct a new DataFrame that counts up how many ratings exist for each movie, and also the average rating while we're at it - that could also come in handy later.

In [47]:
import numpy as np
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movieStats.head()

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

Let's get rid of any movies rated by fewer than 200 people, and check the top-rated ones that are left:

In [50]:
popularMovies = movieStats['rating']['size'] >= 200
popularMovies
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                         size      mean
title                                                  
Schindler's List (1993)                   298  4.466443
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          584  4.359589
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
Godfather, The (1972)                     413  4.283293
Raiders of the Lost Ark (1981)            420  4.252381
Titanic (1997)                            350  4.245714
Empire Strikes Back, The (1980)           368  4.206522
Boot, Das (1981)                          201  4.203980
Godfather: Part II, The (1974)            209  4.186603

200 might still be too low, but these results look pretty good as far as "well rated movies that people have heard of." Let's join this data with our original set of similar movies to Toy Story:

In [51]:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

C:\Users\maisa\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [52]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
2001: A Space Odyssey (1968),259,3.969112,-0.069060
Air Force One (1997),431,3.631090,0.234797
Aladdin (1992),219,3.812785,0.411731
Alien (1979),291,4.034364,0.133508
Aliens (1986),284,3.947183,0.214155


And, sort these new results by similarity score. That's more like it!

In [53]:
df.sort_values(['similarity'], ascending=False)[:20]

,"(rating, size)","(rating, mean)",similarity
title,,,
Toy Story (1995),452,3.878319,1.000000
Beauty and the Beast (1991),202,3.792079,0.442960
"Lion King, The (1994)",220,3.781818,0.426778
Aladdin (1992),219,3.812785,0.411731
Top Gun (1986),220,3.481818,0.384972
Raiders of the Lost Ark (1981),420,4.252381,0.366414
E.T. the Extra-Terrestrial (1982),300,3.833333,0.365349
Jurassic Park (1993),261,3.720307,0.359933
Indiana Jones and the Last Crusade (1989),331,3.930514,0.352870


Ideally we'd also filter out the movie we started from - of course Toy Story is 100% similar to itself. But otherwise these results aren't bad.